# Reading ZRH reports singularily

## 1. The following code is needed to determine at which coordinates the reports from Zurich need to be cut (2010 ZRH is used as a base case)

In [1]:
from tabula import read_pdf
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter, PdfReader
import numpy as np


In [2]:
reader = PdfReader("reduced reports\\reduced Rechnung 2010.pdf", "r")
page = reader.pages[1]

In [3]:
print(page.mediabox) 

RectangleObject([0, 0, 595, 842])


In [4]:
writer = PdfWriter()

In [5]:
for page in range(len(reader.pages)):
    single_page = reader.pages[page]
    single_page.mediabox.upper_right = (537, 750)
    single_page.mediabox.lower_left = (73, 520)
    writer.add_page(single_page)

In [6]:
export = open('2010 ZRH cropped.pdf', 'wb')
writer.write(export)
export.close()

#### The base case coordinates are the following. These values can now be included (hardcoded) in the zurich_data_extraction.py file

In [7]:
(LL_x_base, LL_y_base) = (0, 0)
(UR_x_base, UR_y_base) = (595, 842)

## 2. Now, we read the report with Tabula

In [8]:
df_tabula = read_pdf("cut reports\\cut reduced Rechnung 2010.pdf", pages="all")

In [9]:
# lets analyze two single dfs
print(df_tabula[1].head())

# no matter which of the 4 DFs we take, we need to fix the first row, whose data also belongs to the columns
print(df_tabula[0].columns)

  Verwaltungsrechnung gesamt    R 2009    B 2010    R 2010 Anteil   
0                        NaN       NaN       NaN       NaN    NaN  \
1                 40 Steuern  -2'318.2  -2'247.7  -2'347.1  30.4%   
2   41 Regalien Konzessionen      -5.4      -6.4     -11.2   0.1%   
3        42 Vermögenserträge    -579.5    -344.4    -373.8   4.8%   
4                43 Entgelte  -2'551.0  -2'416.1  -2'498.1  32.4%   

  Abw. zu B 2010  
0      abs. in %  
1      99.4 4.4%  
2      4.8 74.3%  
3      29.3 8.5%  
4      82.0 3.4%  
Index(['R 2009', 'B 2010', 'ZK 2010', 'R 2010', 'Anteil', 'Unnamed: 0'], dtype='object')


#### For some reasons the column "Verwaltungsrechnung gesamt" does not get read by tabula in the first dataframe. In the second dataframe it does always get imported. The reason might be that due to the additional column, which causes the column heading "Verwaltungsrechnung gesamt" to be written in two lines, Tabula sees a potential issue and therefore skips it.

#### This tabula behavior is consistent among all reports between 2010 and 2018 (2019-2022) were skipped, as they do not get read properly by tabula anyway.

#### Knowing this, the values of the column "Verwaltungsrechnung gesamt" for DF1 will be stored in a list and then added to the first df. Noticing that the values of the same column are sometimes spread to different lines, it makes sense to do the same for DF2. 

#### Given that there are always 2 dataframes (due to 2 pages) for every report, we no longer need to loop through all pages of the reduced and cut pdf.

#### Sometimes the last columns also do not get read consistently. As they are not curcial (they can be recalculated if needed), they will be left out. This will be done by chosing to to keep the columns "R {year before}", "B {current year}", "ZK {current year}", "R {current year}", "Anteil". "ZK" only applies to Expenses (df_1), so this has to taken into account for df_2.

In [10]:
missing_col_vals_df_1 = ["30 Personalaufwand", "31 Sachaufwand", "32 Passivzinsen", "33 Abschreibungen", "35 Entschädigungen an Gemeinwesen", "36 Eigene Beiträge", \
                    "37 Durchlaufende Beiträge", "38 Einlagen in Spezialfinanzierungen", "39 Interne Verrechnungen", "Aufwand gesamt"]

In [11]:
missing_col_vals_df_2 = ["40 Steuern", "41 Regalien Konzessionen", "42 Vermögenserträge", "43 Entgelte", "44 Anteile und Beiträge ohne Zweckb.", "45 Rückerstattungen von Gemeinwesen", \
                         "46 Beiträge für eigene Rechnung", "47 Durchlaufende Beiträge", "48 Entnahmen aus Spezialfinanzierungen", "49 Interne Verrechnungen", "Ertrag gesamt neu"]

In [12]:
print(len(missing_col_vals_df_1), len(missing_col_vals_df_2))

10 11


In [13]:
year = int("2010")  # variable of the year, now it is hardcoded but will be extracted from the file name (in the python zurich_data_extraction script)

In [14]:
cols_to_keep = [f"R {str(year - 1)}", f"B {str(year)}", f"ZK {str(year)}", f"R {str(year)}", "Anteil"]

In [15]:
cols_to_keep  # intended use of f strings works

['R 2009', 'B 2010', 'ZK 2010', 'R 2010', 'Anteil']

In [16]:
df_1 = df_tabula[0]
df_2 = df_tabula[1]

In [17]:
df_1

,R 2009,B 2010,ZK 2010,R 2010,Anteil,Unnamed: 0
0,NaN,NaN,NaN,NaN,NaN,abs.
1,NaN,NaN,NaN,NaN,NaN,NaN
2,2'425.8,2'480.2,23.1,2'492.4,32.1%,10.9
3,1'660.9,1'780.5,12.1,1'677.7,21.6%,114.9
4,177.6,181.1,NaN,191.4,2.5%,-10.3
5,600.4,645.2,2.4,585.5,7.5%,62.0
6,190.0,192.4,NaN,194.2,2.5%,-1.8
7,1'263.0,1'312.3,20.0,1'350.9,17.4%,-18.6
8,105.4,95.4,0.3,98.8,1.3%,-3.1
9,565.5,208.7,NaN,235.0,3.0%,-26.3


In [18]:
df_1 = df_1[cols_to_keep] # keeping only the relevant columns

In [19]:
df_1 = df_1.dropna(how='all').reset_index(drop=True)

In [20]:
df_1.insert(0, "Verwaltungsrechnung gesamt", missing_col_vals_df_1)

In [21]:
df_1

,Verwaltungsrechnung gesamt,R 2009,B 2010,ZK 2010,R 2010,Anteil
0,30 Personalaufwand,2'425.8,2'480.2,23.1,2'492.4,32.1%
1,31 Sachaufwand,1'660.9,1'780.5,12.1,1'677.7,21.6%
2,32 Passivzinsen,177.6,181.1,NaN,191.4,2.5%
3,33 Abschreibungen,600.4,645.2,2.4,585.5,7.5%
4,35 Entschädigungen an Gemeinwesen,190.0,192.4,NaN,194.2,2.5%
5,36 Eigene Beiträge,1'263.0,1'312.3,20.0,1'350.9,17.4%
6,37 Durchlaufende Beiträge,105.4,95.4,0.3,98.8,1.3%
7,38 Einlagen in Spezialfinanzierungen,565.5,208.7,NaN,235.0,3.0%
8,39 Interne Verrechnungen,959.8,950.8,1.0,945.0,12.2%
9,Aufwand gesamt,7'948.3,7'846.6,58.9,7'770.9,100.0%


#### df_1 looks good now, up to df_2

In [22]:
df_2

,Verwaltungsrechnung gesamt,R 2009,B 2010,R 2010,Anteil,Abw. zu B 2010
0,NaN,NaN,NaN,NaN,NaN,abs. in %
1,40 Steuern,-2'318.2,-2'247.7,-2'347.1,30.4%,99.4 4.4%
2,41 Regalien Konzessionen,-5.4,-6.4,-11.2,0.1%,4.8 74.3%
3,42 Vermögenserträge,-579.5,-344.4,-373.8,4.8%,29.3 8.5%
4,43 Entgelte,-2'551.0,-2'416.1,-2'498.1,32.4%,82.0 3.4%
5,44 Anteile und Beiträge ohne,-27.2,-26.0,-32.4,0.4%,6.4 24.5%
6,Zweckb.,NaN,NaN,NaN,NaN,NaN
7,45 Rückerstattungen von,-44.1,-45.3,-45.6,0.6%,0.4 0.8%
8,Gemeinwesen,NaN,NaN,NaN,NaN,NaN
9,46 Beiträge für eigene Rechnung,-1'070.7,-1'166.3,-1'145.5,14.8%,-20.8 -1.8%


In [23]:
df_2.insert(3, f"ZK {year}", np.nan)

In [24]:
df_2 = df_2[cols_to_keep] # keeping only the relevant columns

In [25]:
df_2 = df_2.dropna(how='all').reset_index(drop=True)

In [26]:
df_2.insert(0, "Verwaltungsrechnung gesamt", missing_col_vals_df_2)

In [27]:
# concatenating the 2 DFs

df = pd.concat([df_1, df_2])

In [28]:
df

,Verwaltungsrechnung gesamt,R 2009,B 2010,ZK 2010,R 2010,Anteil
0,30 Personalaufwand,2'425.8,2'480.2,23.1,2'492.4,32.1%
1,31 Sachaufwand,1'660.9,1'780.5,12.1,1'677.7,21.6%
2,32 Passivzinsen,177.6,181.1,NaN,191.4,2.5%
3,33 Abschreibungen,600.4,645.2,2.4,585.5,7.5%
4,35 Entschädigungen an Gemeinwesen,190.0,192.4,NaN,194.2,2.5%
5,36 Eigene Beiträge,1'263.0,1'312.3,20.0,1'350.9,17.4%
6,37 Durchlaufende Beiträge,105.4,95.4,0.3,98.8,1.3%
7,38 Einlagen in Spezialfinanzierungen,565.5,208.7,NaN,235.0,3.0%
8,39 Interne Verrechnungen,959.8,950.8,1.0,945.0,12.2%
9,Aufwand gesamt,7'948.3,7'846.6,58.9,7'770.9,100.0%


#### Looks good. Now it can be added in a compact form to zurich_data_extraction.py